In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

1. Read the data file that contains the unit cell vectors, atomic indices, atomic position in fractional coordinates and the hopping parameters in zero potential to pandas dataframe.
2. Pick out the central unit cell (i.e all rows with the unit cell vector = [0,0,0])
3. Pick out the diagonal components i.e. all rows where atom index 1 == atom index 2
4. For these elements, calculate various potential values which varies according to its atomic position.
5. Add the potential value to the existing hopping value of these components.
6. Write these values to TG_hr.dat to be used for bandstructure calculations. Note that we write to a file named 'TG_hr_modified.dat' which lacks the headers and degeneracies that the typical TG_hr.dat file has. Copy these from the TG_hr.dat in zero potential and add to the TG_hr_modified.dat so that wt.in can run without file parsing errors.

The above procedure will be straightforward for a dense matrix since all elements (including zeros) are present in the data file you are reading from in step 1. If you are working with the sparse format AND you set your onsite term to be 0, then the diagonal elements of the central unit cell will be absent in the file in step 1 and the steps following it will not work. The only way around this is to set a very small onsite term like 1e-5 when creating the TG_hr files in zero potential for step 1.

In [24]:
def extract_diagonal_of_central_unit_cell(df):
    return (df['R_vec_x'] == 0) & (df['R_vec_y'] == 0) & (df['R_vec_z'] == 0) & (df['atom1_index'] == df['atom2_index'])

def kronig_penney_potential(potential_length_fractional, potential_height, atom_x_coord_fractional):
    V = -potential_height / 2.0
    if 0 <= atom_x_coord_fractional <= potential_length_fractional:
        V = potential_height / 2.0
    return V

def kronig_penney_potential_break_even_sym(potential_length_fractional, potential_height, atom_x_coord_fractional):
    V = -potential_height / 2.0
    perturb = 0.1 * potential_height / 2.0
    start1, end1 = 0.25, 0.5
    start2, end2 = 0.5, 0.75

    if 0 <= atom_x_coord_fractional <= potential_length_fractional:
        V = potential_height / 2.0
    if start1 <= atom_x_coord_fractional <= end1:
        V += perturb
    if start2 <= atom_x_coord_fractional <= end2:
        V -= perturb

    return V

def kronig_penney_potential_break_odd_sym(potential_length_fractional, potential_height, atom_x_coord_fractional):
    V = -potential_height / 2.0
    perturb = 0.1 * potential_height / 2.0
    start1, end1 = 0.125, 0.375

    if 0 <= atom_x_coord_fractional <= potential_length_fractional:
        V = potential_height / 2.0
    if start1 <= atom_x_coord_fractional <= end1:
        V += perturb

    return V

def sine_potential(potential_height, atom_x_coord_fractional):
    V = potential_height * np.sin(2.0 * np.pi * atom_x_coord_fractional)
    return V

def triangular_potential(potential_height, number_of_wells_factor, atom_x_coord_fractional, atom_y_coord_fractional):
    kx1 = 2 * np.pi / number_of_wells_factor  # number_of_wells_factor can be decreased (increased) to increase (decrease) the number of wells
    ky1 = 2 * np.pi / (number_of_wells_factor * np.sqrt(3.0))

    V = (potential_height / 2) * (np.cos(kx1 * atom_x_coord_fractional) +
                    np.cos(kx1 / 2 * atom_x_coord_fractional + ky1 * atom_y_coord_fractional) +
                    np.cos(kx1 / 2 * atom_x_coord_fractional - ky1 * atom_y_coord_fractional))
    return V

def square_potential(potential_height, atom_x_coord_fractional, atom_y_coord_fractional, number_of_wells_factor=0.15):
    k = 2 * np.pi / number_of_wells_factor
    V = potential_height * (np.sin(k * atom_x_coord_fractional) ** 2 + np.sin(k * atom_y_coord_fractional) ** 2)
    return V

def apply_potential_to_column(df, column_to_update, column_to_apply, potential_func, mask, **kwargs):
    """
    Applies a given potential function to DataFrame columns and adds the result to another column.

    Args:
    df (pd.DataFrame): DataFrame to modify.
    column_to_update (str): Name of the column to update with the result.
    column_to_apply (list): List of column names to apply the potential function to.
    potential_func (function): Potential function to apply.
    mask: Boolean array or condition defining which rows to update.
    **kwargs: Additional keyword arguments to pass to the potential function.
    """
    if len(column_to_apply) == 1:
        potentials = df.loc[mask, column_to_apply[0]].apply(lambda x: potential_func(atom_x_coord_fractional=x, **kwargs))
    else:
        potentials = df.loc[mask, column_to_apply].apply(
            lambda row: potential_func(
                potential_height=kwargs['potential_height'],
                atom_x_coord_fractional=row[column_to_apply[0]],
                atom_y_coord_fractional=row[column_to_apply[1]],
                number_of_wells_factor=kwargs.get('number_of_wells_factor', 0.15)
            ), axis=1)

    df.loc[mask, column_to_update] += potentials
    return df

def process_data_with_potential(filename, potential_func, column_to_update, column_to_apply, **kwargs):
    """
    Loads data, identifies rows to modify, applies a potential, and returns the full DataFrame with modifications applied.
    """
    df = pd.read_csv(filename, header=None, sep='\s+')
    df.columns = [
        'R_vec_x', 'R_vec_y', 'R_vec_z',
        'atom1_index', 'atom2_index',
        'atom1_x_cart', 'atom1_y_cart', 'atom1_z_cart',
        'atom1_x_frac', 'atom1_y_frac', 'atom1_z_frac',
        'hopping_real', 'hopping_imag'
    ]

    modifiable_mask = extract_diagonal_of_central_unit_cell(df)

    updated_df = apply_potential_to_column(
        df=df,
        column_to_update=column_to_update,
        column_to_apply=column_to_apply,
        potential_func=potential_func,
        mask=modifiable_mask,
        **kwargs
    )

    return updated_df

def load_reference_data(filename):
    df = pd.read_csv(filename, header=None, sep='\s+')
    df.columns = [
        'R_vec_x', 'R_vec_y', 'R_vec_z',
        'atom1_index', 'atom2_index',
        'atom1_x_cart', 'atom1_y_cart', 'atom1_z_cart',
        'atom1_x_frac', 'atom1_y_frac', 'atom1_z_frac',
        'hopping_real', 'hopping_imag'
    ]
    return df

def compare_dataframes_with_tolerance(df1, df2, tol=1e-4):
    if not df1.columns.equals(df2.columns):
        return False

    float_cols = df1.select_dtypes(include=[np.float64, np.float32]).columns
    for col in float_cols:
        if not np.allclose(df1[col], df2[col], atol=tol, rtol=0):
            return False

    non_float_cols = df1.select_dtypes(exclude=[np.float64, np.float32]).columns
    if not df1[non_float_cols].equals(df2[non_float_cols]):
        return False

    return True

def write_to_file(output_path,data_with_potential):
    columns_to_write = ['R_vec_x', 'R_vec_y', 'R_vec_z', 'atom1_index', 'atom2_index', 'hopping_real', 'hopping_imag']

    data_to_write = data_with_potential[columns_to_write]


    formatted_data = pd.DataFrame({
    'R_vec_x': data_to_write['R_vec_x'].astype(int).apply('{:>4}'.format),
    'R_vec_y': data_to_write['R_vec_y'].astype(int).apply('{:>4}'.format),
    'R_vec_z': data_to_write['R_vec_z'].astype(int).apply('{:>4}'.format),
    'atom1_index': data_to_write['atom1_index'].astype(int).apply('{:>4}'.format),
    'atom2_index': data_to_write['atom2_index'].astype(int).apply('{:>4}'.format),
    'hopping_real': data_to_write['hopping_real'].apply('{:>20.10E}'.format),
    'hopping_imag': data_to_write['hopping_imag'].apply('{:>20.10E}'.format)
})

    formatted_string = formatted_data.apply(lambda x: ' '.join(x), axis=1)

    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, 'TG_hr_modified.dat'), 'w') as file:
        file.write('\n'.join(formatted_string))
    print("File written successfully with specified format.")

In [25]:
data_path = "graphene/add_potential_externally/kronig/dense"
filename = f"{data_path}/Reference/potential_0/TG_hr_with_atom_coord.dat"

reference_file = f"{data_path}/Reference/potential_10/TG_hr_with_atom_coord.dat"
reference_data = load_reference_data(reference_file)

data_with_potential = process_data_with_potential(
    filename=filename,
    potential_func=kronig_penney_potential,
    column_to_update='hopping_real',
    column_to_apply=['atom1_x_frac'],
    potential_length_fractional=0.5,
    potential_height=10
)

data_identical = data_with_potential.equals(reference_data)
print("Data identical:", data_identical)

output_path = "graphene/add_potential_externally/kronig/dense/potential_10"
write_to_file(output_path,data_with_potential)

Data identical: True
File written successfully with specified format.


In [26]:
data_path = "graphene/add_potential_externally/kronig_break_even/dense"
filename = f"{data_path}/Reference/potential_0/TG_hr_with_atom_coord.dat"

reference_file = f"{data_path}/Reference/potential_10/TG_hr_with_atom_coord.dat"
reference_data = load_reference_data(reference_file)

data_with_potential = process_data_with_potential(
    filename=filename,
    potential_func=kronig_penney_potential_break_even_sym,
    column_to_update='hopping_real',
    column_to_apply=['atom1_x_frac'],
    potential_length_fractional=0.5,
    potential_height=10
)

data_identical = data_with_potential.equals(reference_data)
print("Data identical:", data_identical)

output_path = "graphene/add_potential_externally/kronig_break_even/dense/potential_10"
write_to_file(output_path,data_with_potential)

Data identical: True
File written successfully with specified format.


In [27]:
data_path = "graphene/add_potential_externally/kronig_break_odd/dense"
filename = f"{data_path}/Reference/potential_0/TG_hr_with_atom_coord.dat"

reference_file = f"{data_path}/Reference/potential_10/TG_hr_with_atom_coord.dat"
reference_data = load_reference_data(reference_file)

data_with_potential = process_data_with_potential(
    filename=filename,
    potential_func=kronig_penney_potential_break_odd_sym,
    column_to_update='hopping_real',
    column_to_apply=['atom1_x_frac'],
    potential_length_fractional=0.5,
    potential_height=10
)

data_identical = data_with_potential.equals(reference_data)
print("Data identical:", data_identical)

output_path = "graphene/add_potential_externally/kronig_break_odd/dense/potential_10"
write_to_file(output_path,data_with_potential)

Data identical: True
File written successfully with specified format.


In [28]:
data_path = "graphene/add_potential_externally/sine/dense"
filename = f"{data_path}/Reference/potential_0/TG_hr_with_atom_coord.dat"

reference_file = f"{data_path}/Reference/potential_10/TG_hr_with_atom_coord.dat"
reference_data = load_reference_data(reference_file)
data_with_potential = process_data_with_potential(
    filename=filename,
    potential_func=sine_potential,
    column_to_update='hopping_real',
    column_to_apply=['atom1_x_frac'],
    potential_height=10
)

data_identical = data_with_potential.equals(reference_data)
print("Data identical:", data_identical)

data_identical = compare_dataframes_with_tolerance(data_with_potential, reference_data, tol=0.0001)
print("Data identical with tolerance:", data_identical)

output_path = "graphene/add_potential_externally/sine/dense/potential_10"
write_to_file(output_path,data_with_potential)

Data identical: False
Data identical with tolerance: True
File written successfully with specified format.


In [29]:
data_path = "graphene/add_potential_externally/triangular/dense"
filename = f"{data_path}/Reference/potential_0/TG_hr_with_atom_coord.dat"

reference_file = f"{data_path}/Reference/potential_10/TG_hr_with_atom_coord.dat"
reference_data = load_reference_data(reference_file)

data_with_potential = process_data_with_potential(
    filename=filename,
    potential_func=triangular_potential,
    column_to_update='hopping_real',
    column_to_apply=['atom1_x_frac', 'atom1_y_frac'],
    potential_height=10,
    number_of_wells_factor=0.15
)

data_identical = data_with_potential.equals(reference_data)
print("Data identical:", data_identical)

data_identical = compare_dataframes_with_tolerance(data_with_potential, reference_data, tol=0.001)
print("Data identical with tolerance:", data_identical)

output_path = "graphene/add_potential_externally/triangular/dense/potential_10"
write_to_file(output_path,data_with_potential)

Data identical: False
Data identical with tolerance: True
File written successfully with specified format.
